In [1]:
import os
import fasttext
import fasttext.util
import torch
import math
import random
from early_stopping import EarlyStopping
from sklearn.utils import shuffle
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Unique Naming
from datetime import datetime
import random, string
import importlib

In [2]:
from model import gan

In [3]:
def random_string(length=10):
    """
        Generate a random string of given length. For safely storing produced images.
    """
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))


def get_model_id():
    """
        Creates folder with unique ID in which everything related to a particular testrun can be saved.
    :return: Unique folder identifier
    """
    # Construct testrun identifier
    TIME_STAMP = datetime.now().strftime("%Y_%d_%m__%H_%M_%S__%f_")
    model_folder_id = TIME_STAMP + '_' + random_string() + '/'

    try:
        os.makedirs(model_folder_id)
    except Exception as e:
        print('Exception occurred: ', e)

    return model_folder_id

In [81]:
### VARIABLES & ADMINISTRATIVE STUFF ###
# System
#dataset_path = '/media/daniel/Elements/FastText_Data/'  # In case dataset is stored somewhere else, e.g. on hard-drive
dataset_path = '/media/daniel/Elements/FastText_Data/'  # Data in same directory
dictionary_path = '/media/daniel/Elements/FastText_Data/'  # Dictionaries in same directory
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

# Network
embedding_dim = 300
internal_dim = 300
output_dim = 1

# Train hyperparameters
epochs = 100
batch_size = 32
vocab_size = 10000
num_minibatches = vocab_size // batch_size
real_label, fake_label = 1, 0
languages = {'src': ['de', 'nl']
             , 'trgt': ['en']}  # Target language to be indicated in last position
checkpoint_frequency = 0  # 0 == Off; i > 0 == actual checkpoint frequency in epochs
avg_grads = False  # Boolean indicating whether to average the grads of decoder & discriminator accumulated over nr of source languages by nr of source langs
early_stop = False # Boolean indicating whether to stop early if loss won't decrease for a certain threshold
eval_frequency = 10

#testing parameters
N = [1] # List of n nearest neighbors that will be performed in evaluation


In [5]:
# Changed the way languages are stored. 
# For easy access to complete set of all included languages, just concatenate lists
languages = {'src': ['de', 'nl'], 'trgt': ['en']}

for lang in languages['src']+languages['trgt']:
    print(lang)
print(languages)

de
nl
en
{'src': ['de', 'nl'], 'trgt': ['en']}


In [6]:
# Set up saving paths
data_storage_path = './'
model_id = get_model_id()
checkpoint_path = data_storage_path + model_id + 'Checkpoint/'
final_state_path = data_storage_path + model_id + 'Final/'

try:
    if checkpoint_frequency > 0:
        os.makedirs(checkpoint_path)
        print('Created:', checkpoint_path)
    os.makedirs(final_state_path)
    print('Created:', final_state_path)
except Exception as e:
    raise Warning('Exception occurred: Cound not create dirs! Exception:', e)
    
print('Model ID:', model_id)

Created: ./2020_11_06__12_19_53__629293__jkmwaecdqi/Final/
Model ID: 2020_11_06__12_19_53__629293__jkmwaecdqi/


# Get vocab

In [7]:
def full_vocab(vocab):
    # Returns the word embeddings and matching labels for the full vocabulary
    words = vocab.words
    vectors = [vocab[word] for word in words]
    return vectors, words

In [8]:
def cleaned_vocab(vocab, vocab_size):
    # Remove all punctuation tokens while valid nr of tokens is insufficient yet for having full vocab size
    # TODO & possibly reserve testing vocab
    # Return clean & restricted vocab
    words = vocab.words[:vocab_size]              # Y (labels)
    vects = [vocab[word] for word in words]       # X (input data)

    return vects, words

In [9]:
def add_lang_to_vocab(lang_id, vocab_size, vocabs, full_vocabs):
    # Get dataset
    if dataset_path == './':
        fasttext.util.download_model(lang_id)  # Download word embedding vector data if not available
    vocab = fasttext.load_model(dataset_path + 'cc.' + lang_id + '.300.bin')  # Load language data

    # Add train data (embedding-vectors) and labels (words) to vocab
    X, Y = cleaned_vocab(vocab,500000)
    x, y = cleaned_vocab(vocab, vocab_size)
    vocabs[lang_id] = {'x': torch.tensor(x), 'y': y}
    full_vocabs[lang_id] = {'X': X, 'Y': Y}

    return vocabs, full_vocabs

In [10]:
def load_vocab(languages):
    vocabs = {}
    full_vocabs = {}
    
    for language in languages['src']+languages['trgt']:
        vocabs, full_vocabs = add_lang_to_vocab(language, vocab_size, vocabs, full_vocabs)

    print('Successfully loaded language models.')
    return vocabs, full_vocabs

In [11]:
#load vocab (keep in independent cell for bugfixing purposes)
vocabs, full_vocabs = load_vocab(languages)

Successfully loaded language models.


In [82]:
source_vocabs = {}
source_full_vocabs = {}

for source_language in languages['src']:
    source_vocabs[source_language] = vocabs[source_language]
    source_full_vocabs[source_language] = full_vocabs[source_language]
target_full_vocabs = full_vocabs[languages['trgt'][0]]


# Dictionary related functions

In [83]:
def convert_dictionary(dictionary_text):
    # Converts an input dictionary text file to a python dictionary
    dictionary = {}
    source = True
    source_word = ''
    target_word = ''
    
    for character in dictionary_text:
        if source is True:
            if character is '\t' or character is ' ':
                source = False
            else:
                source_word = source_word + character
        else:
            if character is '\n':
                source = True
                if source_word in dictionary:
                    dictionary[source_word].append(target_word)
                else:
                    dictionary[source_word] = [target_word]
                source_word = ''
                target_word = ''
            else:
                target_word = target_word + character
                
    return dictionary

In [84]:
def load_dictionaries(languages):
    # Loads in the bilingual dictionaries
    dictionaries = {}
    
    for source_language in languages['src']:
        file = open(dictionary_path + source_language + '-' + languages['trgt'][0] + '.txt', 'r', errors='ignore')
        dictionary_text = file.read()
        dictionaries[source_language] = convert_dictionary(dictionary_text)
    
    return dictionaries

In [85]:
def split_translation_task(languages, source_full_vocabs, dictionaries):
    # Creates a split in eval and training translation task
    eval_words = {}
    test_words = {}
    
    for source_language in languages['src']:
        source_words = list(dictionaries[source_language].keys())
#         random.shuffle(source_words)
        eval_list = []
        for source_word in source_words:
            if source_word in source_full_vocabs[source_language]['Y']:
                eval_list.append(source_word)
            if len(eval_list) is 200:
                eval_words[source_language] = eval_list
                break
#             eval_words[source_language] = source_words[0:50]
#             test_words[source_language] = source_words[50:150]        
#         eval_words[source_language] = source_words[0:int(len(source_words)/2)]
#         test_words[source_language] = source_words[int(len(source_words)/2):len(source_words)]
        
    return eval_words, test_words

In [86]:
#load in dictionaries (independent cell for bugfixing)
dictionaries = load_dictionaries(languages)

# split in train and evaluation
eval_words, test_words = split_translation_task(languages, source_full_vocabs, dictionaries)

# Nearest neighbor fitting

In [87]:
def fit_neighbors(N, languages, full_vocabs):
    target_neighbors = {}
    
    for n in N:
        target_neighbors[n] = NearestNeighbors(n_neighbors=n, metric='cosine').fit(full_vocabs[languages['trgt'][0]]['X'])
        
    return target_neighbors

In [88]:
neighbors = fit_neighbors(N, languages, full_vocabs)

# Evaluation functions

In [73]:


def compute_cosine(vector1, vector2):
    # Computes the cosine simularity between two vectors
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    return dot_product/(norm_vector1*norm_vector2)



In [74]:


def get_n_translations_batch(generator, language, source_vectors, target_vocab, neighbors):
    # Gets n possible translations, as given by the n nearest neighbors of the transformed
    # source vector in the target embeddings space, we will use a package for this for optimization
    # purposes. n is given in the nearest neighbor fitting stage.
    #print(source_vector.numpy()[0])
    transformed_source_embedding = generator(torch.as_tensor(source_vectors), language).detach().numpy()

    # only takes 2D arrays, hence the extra bracket [1][0] stands for select indices of
    # the first input vector (the only one in this case)
    vocab_indices = neighbors.kneighbors(np.array(transformed_source_embedding))[1]
    target_vectors = []
    target_words = []
    for target_indices in vocab_indices:
        vectors = []
        words = []
        for index in target_indices:
            vectors.append(target_vocab['X'][index])
            words.append(target_vocab['Y'][index])
        target_vectors.append(vectors)
        target_words.append(words)

    return target_vectors, target_words



In [75]:


def get_average_cosine_batch(generator, language, source_word_vectors, target_vocab, neighbors):
    # Computes the average cosine simularity between the source words and their translations
    sum_of_cosines = 0
    translated_word_vectors = get_n_translations_batch(generator, language, source_word_vectors, target_vocab, neighbors[1])[0]
    for source_word_vector, translated_word_vector in zip(source_word_vectors, translated_word_vectors):
        sum_of_cosines += compute_cosine(source_word_vector, translated_word_vector[0])
    return sum_of_cosines/len(source_word_vectors)



In [90]:


def get_translation_accuracy_batch(generator, language, source_words, source_vocab, target_vocab, dictionary, neighbors):
    # Compute the accuracy of translation over the given set of source words
    correct_translations = 0
    source_word_vectors = []
    for source_word in source_words:
        source_word_index = source_vocab['Y'].index(source_word)
        source_word_vectors.append(source_vocab['X'][source_word_index])
    target_words = get_n_translations_batch(generator, language, source_word_vectors, target_vocab, neighbors)[1]
    print(target_words)
    for n_target_words,source_word in zip(target_words,source_words):
        for target_word in n_target_words:
            #print(target_word,dictionary[source_word]) #check the translation and og word
            if target_word in dictionary[source_word]:
                correct_translations += 1
                break
    return correct_translations/len(source_words)



In [77]:


def evaluation(generator, languages, source_training_vocabs, source_eval_words, source_full_vocabs, target_full_vocabs, dictionaries, neighbors, N):
    # Evaluates the current model by using both an unsupervised cosine similiraty metric and a 
    # supervised translation accuracy metric. We have included both to see how they compare.
    for source_language in languages['src']:
        cosine_metric =  get_average_cosine_batch(generator, source_language, source_training_vocabs[source_language]['x'], target_full_vocabs, neighbors) #experimental
#         cosine_metric = ''
        accuracy_text = 'accuracies are '
        for n in N:
            accuracy = get_translation_accuracy_batch(generator, source_language, source_eval_words[source_language], source_full_vocabs[source_language], target_full_vocabs, dictionaries[source_language], neighbors[n])
            accuracy_text = str(accuracy_text) + 'p@' + str(n) + '=' + str(accuracy) + ', '
        
        print('evaluation of source language ' + source_language + ': average cosine=',cosine_metric, accuracy_text)



In [78]:


def testing(generator, languages, source_test_words, source_full_vocabs, target_full_vocabs, dictionaries, neighbors, N):
    # Testing based on translation accuracy on testing set
    for source_language in languages['src']:
        accuracy_text = ''
        for n in N:
            accuracy = get_translation_accuracy_batch(generator, source_language, source_test_words[source_language], source_full_vocabs[source_language], target_full_vocabs, dictionaries[source_language], neighbors[n])
            accuracy_text = accuracy_text + 'p@' + n + '=' + accuracy + ', '
        
        print('Testing accuracies of source language ' + source_language + ": " + accuracy_text)



# Auto Encoder

In [28]:
class AutoEncoder(torch.nn.Module):

    def __init__(self, D_in, H, shared_decoder):
        # Encoder
        super(AutoEncoder, self).__init__()
        self.w1 = torch.nn.Linear(D_in, H)
        self.w2 = torch.nn.Linear(H, H)
        
        self.activation = torch.nn.functional.relu
        
        # Decoder
        self.decoder = shared_decoder
        
    def forward(self, x):
        # Use encoder for encoding
        x = self.w1(x)
        x = self.activation(x)
        x = self.w2(x)
        
        # Decoder
        x = self.decoder(x)
        
        return x

In [35]:
def AutoEncoderLoss(output, target):
    # MSE loss
    loss = torch.mean((output - target)**2)
    return loss

# Proof of concept of working of Auto Encoder

In [59]:
"""
# Dummy test for AutoEncoder
from model import gan

# Param settings
# Network
embedding_dim = 300
internal_dim = 300
output_dim = 1#2

# Train hyperparameters
epochs = 100
batch_size = 32
vocab_size = 5000
num_minibatches = vocab_size // batch_size
real_label, fake_label = 1, 0
languages = {'src': ['de', 'nl']
             , 'trgt': ['en']}  # Target language to be indicated in last position

# Architecture

net = gan.GAN(embedding_dim, internal_dim, output_dim, languages['src'])

ae = AutoEncoder(embedding_dim, internal_dim, net.generator.decoder)
optimizer_ae = torch.optim.Adam(ae.parameters(),
                                         lr=0.01, betas=(0.9, 0.999), eps=1e-08, 
                                         weight_decay=0., amsgrad=False)

rows, cols = 10, 300
train_data = torch.rand(rows, cols)
print(train_data)


for i in range(1000):
    ae.train()
    optimizer_ae.zero_grad()
    y_pred = ae(train_data)
    loss = AutoEncoderLoss(y_pred, train_data)
    loss.backward()
    optimizer_ae.step()
    print('Epoch', i, 'Loss:', loss.detach().numpy())
"""

tensor([[0.1157, 0.3179, 0.2251,  ..., 0.8784, 0.5264, 0.8604],
        [0.4902, 0.4945, 0.9720,  ..., 0.1604, 0.5067, 0.4805],
        [0.0186, 0.1041, 0.0910,  ..., 0.2663, 0.8779, 0.0575],
        ...,
        [0.5370, 0.3928, 0.5792,  ..., 0.4488, 0.0390, 0.3352],
        [0.9738, 0.7504, 0.3293,  ..., 0.0306, 0.7536, 0.1665],
        [0.3894, 0.9726, 0.0872,  ..., 0.6337, 0.6324, 0.8165]])
Epoch 0 Loss: 0.3368259
Epoch 1 Loss: 1.6825416
Epoch 2 Loss: 2.8612256
Epoch 3 Loss: 0.567075
Epoch 4 Loss: 0.40343323
Epoch 5 Loss: 0.27870342
Epoch 6 Loss: 0.24812168
Epoch 7 Loss: 0.1820921
Epoch 8 Loss: 0.11315657
Epoch 9 Loss: 0.085567564
Epoch 10 Loss: 0.13025013
Epoch 11 Loss: 0.16236624
Epoch 12 Loss: 0.13332967
Epoch 13 Loss: 0.09090811
Epoch 14 Loss: 0.073948815
Epoch 15 Loss: 0.08322731
Epoch 16 Loss: 0.100628905
Epoch 17 Loss: 0.11103383
Epoch 18 Loss: 0.11045902
Epoch 19 Loss: 0.1011927
Epoch 20 Loss: 0.090117045
Epoch 21 Loss: 0.0815799
Epoch 22 Loss: 0.078897156
Epoch 23 Loss: 0.

Epoch 323 Loss: 0.0736981
Epoch 324 Loss: 0.0736981
Epoch 325 Loss: 0.0736981
Epoch 326 Loss: 0.0736981
Epoch 327 Loss: 0.0736981
Epoch 328 Loss: 0.0736981
Epoch 329 Loss: 0.0736981
Epoch 330 Loss: 0.0736981
Epoch 331 Loss: 0.0736981
Epoch 332 Loss: 0.0736981
Epoch 333 Loss: 0.0736981
Epoch 334 Loss: 0.0736981
Epoch 335 Loss: 0.0736981
Epoch 336 Loss: 0.0736981
Epoch 337 Loss: 0.0736981
Epoch 338 Loss: 0.0736981
Epoch 339 Loss: 0.0736981
Epoch 340 Loss: 0.0736981
Epoch 341 Loss: 0.0736981
Epoch 342 Loss: 0.0736981
Epoch 343 Loss: 0.0736981
Epoch 344 Loss: 0.0736981
Epoch 345 Loss: 0.0736981
Epoch 346 Loss: 0.0736981
Epoch 347 Loss: 0.0736981
Epoch 348 Loss: 0.0736981
Epoch 349 Loss: 0.0736981
Epoch 350 Loss: 0.0736981
Epoch 351 Loss: 0.0736981
Epoch 352 Loss: 0.0736981
Epoch 353 Loss: 0.0736981
Epoch 354 Loss: 0.0736981
Epoch 355 Loss: 0.0736981
Epoch 356 Loss: 0.0736981
Epoch 357 Loss: 0.0736981
Epoch 358 Loss: 0.0736981
Epoch 359 Loss: 0.0736981
Epoch 360 Loss: 0.0736981
Epoch 361 Lo

Epoch 643 Loss: 0.0736981
Epoch 644 Loss: 0.0736981
Epoch 645 Loss: 0.0736981
Epoch 646 Loss: 0.0736981
Epoch 647 Loss: 0.0736981
Epoch 648 Loss: 0.0736981
Epoch 649 Loss: 0.0736981
Epoch 650 Loss: 0.0736981
Epoch 651 Loss: 0.0736981
Epoch 652 Loss: 0.0736981
Epoch 653 Loss: 0.0736981
Epoch 654 Loss: 0.0736981
Epoch 655 Loss: 0.0736981
Epoch 656 Loss: 0.0736981
Epoch 657 Loss: 0.0736981
Epoch 658 Loss: 0.0736981
Epoch 659 Loss: 0.0736981
Epoch 660 Loss: 0.0736981
Epoch 661 Loss: 0.0736981
Epoch 662 Loss: 0.0736981
Epoch 663 Loss: 0.0736981
Epoch 664 Loss: 0.0736981
Epoch 665 Loss: 0.0736981
Epoch 666 Loss: 0.0736981
Epoch 667 Loss: 0.0736981
Epoch 668 Loss: 0.0736981
Epoch 669 Loss: 0.0736981
Epoch 670 Loss: 0.0736981
Epoch 671 Loss: 0.0736981
Epoch 672 Loss: 0.0736981
Epoch 673 Loss: 0.0736981
Epoch 674 Loss: 0.0736981
Epoch 675 Loss: 0.0736981
Epoch 676 Loss: 0.0736981
Epoch 677 Loss: 0.0736981
Epoch 678 Loss: 0.0736981
Epoch 679 Loss: 0.0736981
Epoch 680 Loss: 0.0736981
Epoch 681 Lo

Epoch 977 Loss: 0.0736981
Epoch 978 Loss: 0.0736981
Epoch 979 Loss: 0.0736981
Epoch 980 Loss: 0.0736981
Epoch 981 Loss: 0.0736981
Epoch 982 Loss: 0.0736981
Epoch 983 Loss: 0.0736981
Epoch 984 Loss: 0.0736981
Epoch 985 Loss: 0.0736981
Epoch 986 Loss: 0.0736981
Epoch 987 Loss: 0.0736981
Epoch 988 Loss: 0.0736981
Epoch 989 Loss: 0.0736981
Epoch 990 Loss: 0.0736981
Epoch 991 Loss: 0.0736981
Epoch 992 Loss: 0.0736981
Epoch 993 Loss: 0.0736981
Epoch 994 Loss: 0.0736981
Epoch 995 Loss: 0.0736981
Epoch 996 Loss: 0.0736981
Epoch 997 Loss: 0.0736981
Epoch 998 Loss: 0.0736981
Epoch 999 Loss: 0.0736981


# NoisyAutoEncoder to fight Mode Collapse

In [29]:
def bound_noise(shape, min_=-0.5, max_=0.5):
    return (min_ - max_) * torch.rand(shape) + max_

In [30]:
class NoisyAutoEncoder(torch.nn.Module):

    def __init__(self, shared_encoder, D_in, H, D_out):
        super(NoisyAutoEncoder, self).__init__()
        # Encoder
        self.encoder = shared_encoder
        
        # Decoder
        self.w1 = torch.nn.Linear(D_in, H)
        self.w2 = torch.nn.Linear(H, H)
        
        self.activation = torch.nn.functional.relu
        
        self.noise = bound_noise
        
    def forward(self, x):
        # Use encoder for encoding, but reproduce original encoder input in the presence of noise
        # Encoder
        x = self.encoder(x)
        
        # Noisy Decoder
        x = x + self.noise(x.shape)
        x = self.w1(x)
        x = x + self.noise(x.shape)
        x = self.activation(x)
        x = self.w2(x)
        
        return x

In [38]:
"""
# Dummy test for NoisyAutoEncoder
from model import gan

# Param settings
# Network
embedding_dim = 300
internal_dim = 300
output_dim = 1

# Train hyperparameters
epochs = 100

languages = {'src': ['de', 'nl']
             , 'trgt': ['en']}  # Target language to be indicated in last position

# Architectures
net = gan.GAN(embedding_dim, internal_dim, output_dim, languages['src'])

nae_de = NoisyAutoEncoder(net.generator.encoders['de'], embedding_dim, internal_dim, embedding_dim)
optimizer_nae_de = torch.optim.Adam(nae_de.parameters(),
                                         lr=0.01, betas=(0.9, 0.999), eps=1e-08, 
                                         weight_decay=0., amsgrad=False)

# Generate dummy train data
rows, cols = 10, 300
train_data = torch.rand(rows, cols)
print(train_data)

# Training
for i in range(1000):
    
    nae_de.train()
    optimizer_nae_de.zero_grad()
    y_pred = nae_de(train_data)
    loss = AutoEncoderLoss(y_pred, train_data)
    loss.backward()
    optimizer_nae_de.step()
    print('Epoch', i, 'Loss:', loss.detach().numpy())
"""

tensor([[0.9900, 0.2840, 0.5615,  ..., 0.1994, 0.3882, 0.8703],
        [0.1917, 0.3195, 0.7581,  ..., 0.1159, 0.2907, 0.7473],
        [0.5165, 0.6211, 0.7069,  ..., 0.2440, 0.6162, 0.4669],
        ...,
        [0.1462, 0.2985, 0.7205,  ..., 0.5906, 0.8252, 0.9475],
        [0.9766, 0.6510, 0.4914,  ..., 0.0468, 0.8069, 0.9557],
        [0.1095, 0.2054, 0.6183,  ..., 0.4334, 0.8088, 0.8357]])
Epoch 0 Loss: 0.3520835
Epoch 1 Loss: 1.2394924
Epoch 2 Loss: 0.13039304
Epoch 3 Loss: 0.19542731
Epoch 4 Loss: 0.15765649
Epoch 5 Loss: 0.101449646
Epoch 6 Loss: 0.14557031
Epoch 7 Loss: 0.094931275
Epoch 8 Loss: 0.10400712
Epoch 9 Loss: 0.094734505
Epoch 10 Loss: 0.11290293
Epoch 11 Loss: 0.13206723
Epoch 12 Loss: 0.10016042
Epoch 13 Loss: 0.107948214
Epoch 14 Loss: 0.10394173
Epoch 15 Loss: 0.11061392
Epoch 16 Loss: 0.09632213
Epoch 17 Loss: 0.10563476
Epoch 18 Loss: 0.11406905
Epoch 19 Loss: 0.09737422
Epoch 20 Loss: 0.09906917
Epoch 21 Loss: 0.099579945
Epoch 22 Loss: 0.1165018
Epoch 23 Los

Epoch 293 Loss: 0.07964518
Epoch 294 Loss: 0.07738579
Epoch 295 Loss: 0.07720961
Epoch 296 Loss: 0.077539675
Epoch 297 Loss: 0.07839741
Epoch 298 Loss: 0.07528335
Epoch 299 Loss: 0.07724279
Epoch 300 Loss: 0.07727555
Epoch 301 Loss: 0.07786457
Epoch 302 Loss: 0.07576873
Epoch 303 Loss: 0.07783124
Epoch 304 Loss: 0.07750172
Epoch 305 Loss: 0.07633712
Epoch 306 Loss: 0.077037245
Epoch 307 Loss: 0.07726361
Epoch 308 Loss: 0.07700386
Epoch 309 Loss: 0.075939715
Epoch 310 Loss: 0.07589312
Epoch 311 Loss: 0.0749395
Epoch 312 Loss: 0.077648774
Epoch 313 Loss: 0.07578706
Epoch 314 Loss: 0.07697366
Epoch 315 Loss: 0.07599789
Epoch 316 Loss: 0.07853657
Epoch 317 Loss: 0.07668741
Epoch 318 Loss: 0.077112414
Epoch 319 Loss: 0.07695548
Epoch 320 Loss: 0.07754452
Epoch 321 Loss: 0.080005206
Epoch 322 Loss: 0.07755538
Epoch 323 Loss: 0.077004746
Epoch 324 Loss: 0.07913999
Epoch 325 Loss: 0.076530635
Epoch 326 Loss: 0.07828965
Epoch 327 Loss: 0.07742887
Epoch 328 Loss: 0.07983562
Epoch 329 Loss: 0.075

Epoch 608 Loss: 0.07754671
Epoch 609 Loss: 0.07682147
Epoch 610 Loss: 0.07614337
Epoch 611 Loss: 0.07723021
Epoch 612 Loss: 0.075937115
Epoch 613 Loss: 0.07527121
Epoch 614 Loss: 0.07576386
Epoch 615 Loss: 0.07562368
Epoch 616 Loss: 0.07635725
Epoch 617 Loss: 0.07522703
Epoch 618 Loss: 0.07632688
Epoch 619 Loss: 0.076196216
Epoch 620 Loss: 0.07561488
Epoch 621 Loss: 0.076456115
Epoch 622 Loss: 0.07698342
Epoch 623 Loss: 0.07500197
Epoch 624 Loss: 0.076182805
Epoch 625 Loss: 0.07684878
Epoch 626 Loss: 0.07328678
Epoch 627 Loss: 0.07525389
Epoch 628 Loss: 0.07596387
Epoch 629 Loss: 0.07579768
Epoch 630 Loss: 0.075109236
Epoch 631 Loss: 0.07753661
Epoch 632 Loss: 0.076400734
Epoch 633 Loss: 0.0753951
Epoch 634 Loss: 0.07385521
Epoch 635 Loss: 0.07658775
Epoch 636 Loss: 0.07674189
Epoch 637 Loss: 0.07766628
Epoch 638 Loss: 0.07671711
Epoch 639 Loss: 0.07597279
Epoch 640 Loss: 0.07708829
Epoch 641 Loss: 0.078816384
Epoch 642 Loss: 0.07667414
Epoch 643 Loss: 0.07614332
Epoch 644 Loss: 0.0760

Epoch 913 Loss: 0.07640025
Epoch 914 Loss: 0.07539837
Epoch 915 Loss: 0.07695417
Epoch 916 Loss: 0.076285236
Epoch 917 Loss: 0.07587158
Epoch 918 Loss: 0.07722051
Epoch 919 Loss: 0.07687733
Epoch 920 Loss: 0.07568272
Epoch 921 Loss: 0.074942194
Epoch 922 Loss: 0.07578083
Epoch 923 Loss: 0.074857175
Epoch 924 Loss: 0.07690501
Epoch 925 Loss: 0.07403846
Epoch 926 Loss: 0.07534805
Epoch 927 Loss: 0.0774805
Epoch 928 Loss: 0.075288095
Epoch 929 Loss: 0.076457754
Epoch 930 Loss: 0.07649968
Epoch 931 Loss: 0.07547515
Epoch 932 Loss: 0.07647244
Epoch 933 Loss: 0.078840524
Epoch 934 Loss: 0.07836341
Epoch 935 Loss: 0.07503583
Epoch 936 Loss: 0.07713639
Epoch 937 Loss: 0.075581245
Epoch 938 Loss: 0.07535679
Epoch 939 Loss: 0.075398475
Epoch 940 Loss: 0.075459935
Epoch 941 Loss: 0.076232724
Epoch 942 Loss: 0.07798584
Epoch 943 Loss: 0.07710821
Epoch 944 Loss: 0.07730417
Epoch 945 Loss: 0.07839004
Epoch 946 Loss: 0.07750914
Epoch 947 Loss: 0.07757714
Epoch 948 Loss: 0.07706348
Epoch 949 Loss: 0.0

In [19]:
"""
x = torch.tensor([[0.1,1.],[0.1,1.],[0.1,1.]])
noise1 = torch.rand(x.shape)
print(noise1)

def bound_noise(shape, min_=-0.3, max_=0.3):
    return (min_ - max_) * torch.rand(shape) + max_

f =  bound_noise

f(x.shape)
"""

tensor([[0.6685, 0.8666],
        [0.9412, 0.3197],
        [0.4982, 0.7416]])


tensor([[-0.2810,  0.2586],
        [-0.2750,  0.2853],
        [-0.1792,  0.0030]])

# Define training related functions

In [40]:
def save_checkpoint(data, save):
    if save:
        torch.save(data, checkpoint_path + 'checkpoint_%d.pt' % data['epoch'])

In [41]:
def mean_param(model):
    return torch.mean(torch.cat([param.data.view(-1) for param in model.parameters()], 0))

In [42]:
def get_dataset_sample(lang, vocab, batch_size, include_y=False):
    """
    This function draws batch_size-many training samples at random 
    from a vocab corresponding to queried language.  
    """
    indices = torch.LongTensor(batch_size).random_(0, len(vocab))
    if include_y:
        return vocab['x'][indices], vocab['y'][indices]
    return vocab['x'][indices]


def get_train_data(languages, vocabs, batch_size, include_y=False):
    """
    Returns one set of samples datapoints form a vocabulary for each provided language.
    """
    x, y = {}, {}
    
    # Source languages
    for lang in languages['src']+languages['trgt']:
        if include_y:
            x[lang], y[lang] = get_dataset_sample(lang, vocabs[lang], batch_size, include_y)
        else:
            x[lang] = get_dataset_sample(lang, vocabs[lang], batch_size)
    
    # Return
    if include_y:
        return x, y
    return x

In [43]:
# For debugging - Compute sum of abs(gradients) of model
def get_summed_abs_grads(model):
#     summed_abs = torch.tensor(0)
    summed_abs = 0
    for p in model.parameters():
        summed_abs += torch.sum(torch.abs(p))
    return summed_abs

In [43]:
"""
model = net = gan.GAN(embedding_dim, internal_dim, output_dim, languages['src'])

print(model.generator.parameters())
print(model.discriminator.parameters())
print(mean_param(model.generator))
print(mean_param(model.discriminator))
print(model.generator)
print(model.discriminator)
"""

<generator object Module.parameters at 0x7f11e6f5d048>
<generator object Module.parameters at 0x7f11e6f5d048>
tensor(-8.5159e-05)
tensor(5.7731e-05)
Generator(
  (decoder): FeedForwardDecoder(
    (w1): Linear(in_features=300, out_features=300, bias=True)
    (w2): Linear(in_features=300, out_features=300, bias=True)
  )
)
Discriminator(
  (w1): Linear(in_features=300, out_features=2048, bias=True)
  (w2): Linear(in_features=2048, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


# Proof of concept of working of GAN

In [58]:
"""
rows, cols = 8, 5
batch_size_1 = rows
train_data = torch.rand(rows, cols)
print(train_data)

embedding_dim, internal_dim, output_dim = cols, 30, 1

langs = {'src': ['de'], 'trgt': 'en'}
nr_langs = 2

### GAN ###
importlib.reload(gan)

# Set up model architecture
net = gan.GAN(embedding_dim, internal_dim, output_dim, langs['src'])
print(net)

criterion = torch.nn.BCELoss()

print('Train data:')
print(train_data)
print('Translations:')
print(net.generator(train_data, 'de'))
print('Before:', AutoEncoderLoss(train_data, net.generator(train_data, 'de')))

# Get optimizers; 1 per source language of encoder and 1 for discriminator
optimizers = {'gen': {}}
for lang in langs['src']:
    optimizers['gen'][lang] = torch.optim.Adam([{'params': net.generator.encoders[lang].parameters()},
                                                {'params': net.generator.decoder.parameters()}
                                               ],
                                                lr=0.0001, betas=(0.9, 0.999), eps=1e-08, 
                                                weight_decay=0.01, amsgrad=False)
optimizers['dis'] = torch.optim.Adam(net.discriminator.parameters(),
                                     lr=0.001, betas=(0.9, 0.999), eps=1e-08, 
                                     weight_decay=0.001, amsgrad=False)


# Train
train_loss_gen, train_loss_dis, train_loss_ae = [], [], []
eval_loss = [] # TODO: To be populated...
last_loss = -1


for epoch in range(10000):
    print('Epoch ', epoch, '/', epochs)
    loss_gen, loss_dis, loss_ae = 0., 0., 0.

    # Train #

    ### GAN STEP ###
    # Update discriminator #
    net.discriminator.train()
    net.generator.eval()
    net.discriminator.zero_grad()

    # y_true
    y_true = torch.zeros([nr_langs*batch_size_1])
    y_true[0:batch_size_1] = real_label  # First elements are target embeddings


    # All-real minibatch
    y_preds = torch.zeros([nr_langs*batch_size_1, 1]) 
    y_preds[0:batch_size_1] = net.discriminator(train_data)

    # All-fake minibatches - One minibatch per source language
    x_trans = net.generator(train_data, 'de')  # Generate fake data aka translate
    y_preds[batch_size_1:] = net.discriminator(x_trans)

    #print('Preds:', y_preds)

    # Loss proportional to discriminator's probability of correctly distinguishing TP and FP
    loss = criterion(y_preds, y_true)
    loss.backward()    # Compute gradients only for discriminator
    optimizers['dis'].step() 
    print('Dis loss:', loss.detach().numpy())


    # Update generator #
    net.generator.train()
    net.discriminator.eval()
    net.generator.zero_grad()

    # All-fake minibatches - One minibatch per source language
    y_true = torch.full((batch_size_1,), real_label)
    x_trans = net.generator(train_data, 'de')
    y_pred = net.discriminator(x_trans)
    # Loss proportional to discriminator's probability of misclassifying TP and FP
    loss = criterion(y_pred, y_true)
    loss.backward()    # Compute gradients only for discriminator
    print('Gen loss:', loss.detach().numpy())


    # Perform weight updates
    optimizers['gen']['de'].step()

    print('Summed abs weights Generator:', get_summed_abs_grads(net.generator).detach().numpy())
    print('Summed abs weights Discrimi.:', get_summed_abs_grads(net.discriminator).detach().numpy())
    
print('Train data:')
print(train_data)
print('Translations:')
print(net.generator(train_data, 'de'))
print('After:', AutoEncoderLoss(train_data, net.generator(train_data, 'de')))
"""

tensor([[0.3777, 0.7403, 0.8535, 0.3279, 0.0729],
        [0.5718, 0.3703, 0.3002, 0.5289, 0.4503],
        [0.7872, 0.7883, 0.1095, 0.7093, 0.4496],
        [0.1656, 0.2281, 0.6828, 0.1863, 0.4474],
        [0.7573, 0.1110, 0.5363, 0.6793, 0.0377],
        [0.6993, 0.2236, 0.0127, 0.6137, 0.6126],
        [0.4725, 0.4378, 0.6787, 0.6610, 0.4836],
        [0.2982, 0.3069, 0.1087, 0.9310, 0.4253]])
GAN(
  (generator): Generator(
    (decoder): FeedForwardDecoder(
      (w1): Linear(in_features=30, out_features=30, bias=True)
      (w2): Linear(in_features=30, out_features=5, bias=True)
    )
  )
  (discriminator): Discriminator(
    (w1): Linear(in_features=5, out_features=2048, bias=True)
    (w2): Linear(in_features=2048, out_features=1, bias=True)
    (sigmoid): Sigmoid()
  )
)
Train data:
tensor([[0.3777, 0.7403, 0.8535, 0.3279, 0.0729],
        [0.5718, 0.3703, 0.3002, 0.5289, 0.4503],
        [0.7872, 0.7883, 0.1095, 0.7093, 0.4496],
        [0.1656, 0.2281, 0.6828, 0.1863, 0.4474

Dis loss: 0.023941942
Gen loss: 3.6075842
Summed abs weights Generator: 93.67315
Summed abs weights Discrimi.: 2282.8442
Epoch  84 / 100
Dis loss: 0.023876734
Gen loss: 3.6098974
Summed abs weights Generator: 93.648224
Summed abs weights Discrimi.: 2276.4883
Epoch  85 / 100
Dis loss: 0.023814686
Gen loss: 3.612191
Summed abs weights Generator: 93.62338
Summed abs weights Discrimi.: 2270.1448
Epoch  86 / 100
Dis loss: 0.023757037
Gen loss: 3.6143243
Summed abs weights Generator: 93.59862
Summed abs weights Discrimi.: 2263.8147
Epoch  87 / 100
Dis loss: 0.02370703
Gen loss: 3.6161232
Summed abs weights Generator: 93.57395
Summed abs weights Discrimi.: 2257.504
Epoch  88 / 100
Dis loss: 0.02366251
Gen loss: 3.6176307
Summed abs weights Generator: 93.549355
Summed abs weights Discrimi.: 2251.2078
Epoch  89 / 100
Dis loss: 0.023620669
Gen loss: 3.619092
Summed abs weights Generator: 93.52488
Summed abs weights Discrimi.: 2244.9258
Epoch  90 / 100
Dis loss: 0.02358349
Gen loss: 3.6203954
Sum

Summed abs weights Generator: 92.5186
Summed abs weights Discrimi.: 1806.8506
Epoch  173 / 100
Dis loss: 0.040461734
Gen loss: 3.2807636
Summed abs weights Generator: 92.51675
Summed abs weights Discrimi.: 1802.9703
Epoch  174 / 100
Dis loss: 0.041407526
Gen loss: 3.264545
Summed abs weights Generator: 92.51493
Summed abs weights Discrimi.: 1799.1399
Epoch  175 / 100
Dis loss: 0.042410336
Gen loss: 3.2468345
Summed abs weights Generator: 92.513245
Summed abs weights Discrimi.: 1795.3616
Epoch  176 / 100
Dis loss: 0.043462705
Gen loss: 3.227938
Summed abs weights Generator: 92.51169
Summed abs weights Discrimi.: 1791.644
Epoch  177 / 100
Dis loss: 0.044560395
Gen loss: 3.20809
Summed abs weights Generator: 92.51023
Summed abs weights Discrimi.: 1787.9751
Epoch  178 / 100
Dis loss: 0.04572343
Gen loss: 3.1870067
Summed abs weights Generator: 92.50891
Summed abs weights Discrimi.: 1784.3538
Epoch  179 / 100
Dis loss: 0.046967845
Gen loss: 3.1652408
Summed abs weights Generator: 92.507706


Gen loss: 1.7131677
Summed abs weights Generator: 92.642166
Summed abs weights Discrimi.: 1496.1975
Epoch  263 / 100
Dis loss: 0.37015584
Gen loss: 1.7227993
Summed abs weights Generator: 92.63547
Summed abs weights Discrimi.: 1491.5286
Epoch  264 / 100
Dis loss: 0.36196268
Gen loss: 1.7317088
Summed abs weights Generator: 92.62832
Summed abs weights Discrimi.: 1486.9208
Epoch  265 / 100
Dis loss: 0.3536433
Gen loss: 1.7416059
Summed abs weights Generator: 92.620735
Summed abs weights Discrimi.: 1482.3453
Epoch  266 / 100
Dis loss: 0.34523156
Gen loss: 1.754168
Summed abs weights Generator: 92.612724
Summed abs weights Discrimi.: 1477.816
Epoch  267 / 100
Dis loss: 0.33672437
Gen loss: 1.7693142
Summed abs weights Generator: 92.60428
Summed abs weights Discrimi.: 1473.3163
Epoch  268 / 100
Dis loss: 0.3280661
Gen loss: 1.7854786
Summed abs weights Generator: 92.59544
Summed abs weights Discrimi.: 1468.8569
Epoch  269 / 100
Dis loss: 0.31927818
Gen loss: 1.8004987
Summed abs weights Gen

Gen loss: 3.3513157
Summed abs weights Generator: 91.194176
Summed abs weights Discrimi.: 1286.3632
Epoch  341 / 100
Dis loss: 0.05267732
Gen loss: 3.3521552
Summed abs weights Generator: 91.16866
Summed abs weights Discrimi.: 1284.4333
Epoch  342 / 100
Dis loss: 0.052781414
Gen loss: 3.351799
Summed abs weights Generator: 91.14299
Summed abs weights Discrimi.: 1282.5015
Epoch  343 / 100
Dis loss: 0.05292287
Gen loss: 3.3503828
Summed abs weights Generator: 91.11723
Summed abs weights Discrimi.: 1280.575
Epoch  344 / 100
Dis loss: 0.053099927
Gen loss: 3.3480473
Summed abs weights Generator: 91.09135
Summed abs weights Discrimi.: 1278.6503
Epoch  345 / 100
Dis loss: 0.053316806
Gen loss: 3.3444247
Summed abs weights Generator: 91.06537
Summed abs weights Discrimi.: 1276.728
Epoch  346 / 100
Dis loss: 0.05356076
Gen loss: 3.3398619
Summed abs weights Generator: 91.03928
Summed abs weights Discrimi.: 1274.8054
Epoch  347 / 100
Dis loss: 0.053828612
Gen loss: 3.336747
Summed abs weights G

...

Dis loss: 0.32612202
Gen loss: 1.4428825
Summed abs weights Generator: 71.91763
Summed abs weights Discrimi.: 845.601
Epoch  9623 / 100
Dis loss: 0.3268475
Gen loss: 1.4428726
Summed abs weights Generator: 71.92111
Summed abs weights Discrimi.: 845.47614
Epoch  9624 / 100
Dis loss: 0.3278537
Gen loss: 1.4434237
Summed abs weights Generator: 71.92473
Summed abs weights Discrimi.: 845.34906
Epoch  9625 / 100
Dis loss: 0.3290429
Gen loss: 1.443941
Summed abs weights Generator: 71.92849
Summed abs weights Discrimi.: 845.215
Epoch  9626 / 100
Dis loss: 0.3304265
Gen loss: 1.4425714
Summed abs weights Generator: 71.93236
Summed abs weights Discrimi.: 845.0722
Epoch  9627 / 100
Dis loss: 0.33198422
Gen loss: 1.440365
Summed abs weights Generator: 71.93632
Summed abs weights Discrimi.: 844.91626
Epoch  9628 / 100
Dis loss: 0.3338263
Gen loss: 1.4363546
Summed abs weights Generator: 71.9405
Summed abs weights Discrimi.: 844.7477
Epoch  9629 / 100
Dis loss: 0.33588687
Gen loss: 1.4288001
Summed 

Gen loss: 1.2359236
Summed abs weights Generator: 72.294075
Summed abs weights Discrimi.: 820.84406
Epoch  9699 / 100
Dis loss: 0.40278065
Gen loss: 1.2462025
Summed abs weights Generator: 72.29356
Summed abs weights Discrimi.: 820.92194
Epoch  9700 / 100
Dis loss: 0.39854342
Gen loss: 1.2560476
Summed abs weights Generator: 72.29268
Summed abs weights Discrimi.: 821.0157
Epoch  9701 / 100
Dis loss: 0.39449227
Gen loss: 1.2655593
Summed abs weights Generator: 72.29143
Summed abs weights Discrimi.: 821.1189
Epoch  9702 / 100
Dis loss: 0.39042825
Gen loss: 1.2737075
Summed abs weights Generator: 72.289795
Summed abs weights Discrimi.: 821.2375
Epoch  9703 / 100
Dis loss: 0.3863573
Gen loss: 1.2809609
Summed abs weights Generator: 72.287834
Summed abs weights Discrimi.: 821.36584
Epoch  9704 / 100
Dis loss: 0.3821905
Gen loss: 1.2870033
Summed abs weights Generator: 72.28551
Summed abs weights Discrimi.: 821.5006
Epoch  9705 / 100
Dis loss: 0.3780192
Gen loss: 1.2933364
Summed abs weights

Gen loss: 1.1572915
Summed abs weights Generator: 71.835075
Summed abs weights Discrimi.: 810.4606
Epoch  9774 / 100
Dis loss: 0.4619072
Gen loss: 1.1601257
Summed abs weights Generator: 71.83163
Summed abs weights Discrimi.: 810.2069
Epoch  9775 / 100
Dis loss: 0.45703787
Gen loss: 1.1611578
Summed abs weights Generator: 71.828514
Summed abs weights Discrimi.: 809.993
Epoch  9776 / 100
Dis loss: 0.45222118
Gen loss: 1.164536
Summed abs weights Generator: 71.82557
Summed abs weights Discrimi.: 809.82135
Epoch  9777 / 100
Dis loss: 0.44761625
Gen loss: 1.1701117
Summed abs weights Generator: 71.823006
Summed abs weights Discrimi.: 809.6904
Epoch  9778 / 100
Dis loss: 0.44310486
Gen loss: 1.1802164
Summed abs weights Generator: 71.82073
Summed abs weights Discrimi.: 809.5991
Epoch  9779 / 100
Dis loss: 0.43828136
Gen loss: 1.1909623
Summed abs weights Generator: 71.81866
Summed abs weights Discrimi.: 809.53064
Epoch  9780 / 100
Dis loss: 0.4330885
Gen loss: 1.2002678
Summed abs weights G

Gen loss: 1.1709815
Summed abs weights Generator: 72.14785
Summed abs weights Discrimi.: 809.02014
Epoch  9851 / 100
Dis loss: 0.4970985
Gen loss: 1.1708869
Summed abs weights Generator: 72.15604
Summed abs weights Discrimi.: 808.7388
Epoch  9852 / 100
Dis loss: 0.49766827
Gen loss: 1.1688938
Summed abs weights Generator: 72.16405
Summed abs weights Discrimi.: 808.4541
Epoch  9853 / 100
Dis loss: 0.49773854
Gen loss: 1.1635463
Summed abs weights Generator: 72.1719
Summed abs weights Discrimi.: 808.1729
Epoch  9854 / 100
Dis loss: 0.49761224
Gen loss: 1.1622043
Summed abs weights Generator: 72.1795
Summed abs weights Discrimi.: 807.8943
Epoch  9855 / 100
Dis loss: 0.49655077
Gen loss: 1.1650767
Summed abs weights Generator: 72.186806
Summed abs weights Discrimi.: 807.6016
Epoch  9856 / 100
Dis loss: 0.494668
Gen loss: 1.1681054
Summed abs weights Generator: 72.19378
Summed abs weights Discrimi.: 807.2963
Epoch  9857 / 100
Dis loss: 0.49163964
Gen loss: 1.1697446
Summed abs weights Gener

Summed abs weights Discrimi.: 812.64594
Epoch  9927 / 100
Dis loss: 0.47431937
Gen loss: 1.2262534
Summed abs weights Generator: 71.703545
Summed abs weights Discrimi.: 812.552
Epoch  9928 / 100
Dis loss: 0.48082963
Gen loss: 1.2186344
Summed abs weights Generator: 71.690155
Summed abs weights Discrimi.: 812.4321
Epoch  9929 / 100
Dis loss: 0.48691905
Gen loss: 1.2085632
Summed abs weights Generator: 71.676865
Summed abs weights Discrimi.: 812.2875
Epoch  9930 / 100
Dis loss: 0.49291763
Gen loss: 1.1970494
Summed abs weights Generator: 71.663704
Summed abs weights Discrimi.: 812.1326
Epoch  9931 / 100
Dis loss: 0.49857485
Gen loss: 1.1832744
Summed abs weights Generator: 71.65064
Summed abs weights Discrimi.: 811.9572
Epoch  9932 / 100
Dis loss: 0.50390494
Gen loss: 1.1674615
Summed abs weights Generator: 71.63773
Summed abs weights Discrimi.: 811.74774
Epoch  9933 / 100
Dis loss: 0.50877774
Gen loss: 1.1575613
Summed abs weights Generator: 71.62502
Summed abs weights Discrimi.: 811.51

# Main algo

In [ ]:
def main():
    
    nr_src_langs = len(languages['src'])
    nr_trgt_langs = len(languages['trgt'])
    nr_langs = nr_src_langs + nr_trgt_langs
    print('Nr source languages:', nr_src_langs)
    print('Nr target languages:', len(languages['trgt'])) 
    print('\n', languages)
    
    #if avg_grads:
    #    avg_factor = 1/nr_src_langs
    #    print('Decoder gradient averaging factor:', avg_factor, "\n")
    
    ### GAN ###
    importlib.reload(gan)
    
    # Set up model architecture
    net = gan.GAN(embedding_dim, internal_dim, output_dim, languages['src'])
    print(net)
    #NLLLoss = torch.nn.NLLLoss()
    criterion = torch.nn.BCELoss()

    # Get optimizers; 1 per source language of encoder and 1 for discriminator
    optimizers = {'gen': {}}
    for lang in languages['src']:
        optimizers['gen'][lang] = torch.optim.Adam([{'params': net.generator.encoders[lang].parameters()},
                                                    {'params': net.generator.decoder.parameters()}
                                                   ],
                                                    lr=0.0001, betas=(0.9, 0.999), eps=1e-08, 
                                                    weight_decay=0.01, amsgrad=False)
    optimizers['dis'] = torch.optim.Adam(net.discriminator.parameters(),
                                         lr=0.001, betas=(0.9, 0.999), eps=1e-08, 
                                         weight_decay=0.001, amsgrad=False)
    
    ### AUTO ENCODER ###
    ae = AutoEncoder(embedding_dim, internal_dim, net.generator.decoder)
    optimizer_ae = torch.optim.Adam(ae.parameters(),
                                         lr=0.00001, betas=(0.9, 0.999), eps=1e-08, 
                                         weight_decay=0., amsgrad=False)
    
    
    ### NOISY AUTO ENCODER ###
    nae = {}
    optimizers_nae =  {}
    for lang in languages['src']:
        nae[lang] = NoisyAutoEncoder(net.generator.encoders[lang], embedding_dim, internal_dim, embedding_dim)
        optimizers_nae[lang] = torch.optim.Adam(nae[lang].parameters(),
                                             lr=0.00001, betas=(0.9, 0.999), eps=1e-08, 
                                             weight_decay=0., amsgrad=False)
    
    # Train
    train_loss_gen, train_loss_dis, train_loss_ae, train_loss_nae = [], [], [], []
    eval_loss = [] # TODO: To be populated...
    last_loss = -1
    
    es = EarlyStopping(patience=10) #patience = amount of epochs the loss has to stop decreasing in a row for it to early stop
    
    for epoch in range(epochs):
        print('Epoch ', epoch, '/', epochs)
        loss_gen, loss_dis, loss_ae, loss_nae = 0., 0., 0., 0.

        # Train #
        for batch in range(num_minibatches):
            
            ### GAN STEP ###
            # Update discriminator #
            net.discriminator.train()
            net.generator.eval()
            net.discriminator.zero_grad()
            
            # Retrieve data
            x = get_train_data(languages, vocabs, batch_size)#.to(device) 

            # Init data-storage
            y_preds = torch.zeros([nr_langs*batch_size, 1]) # 2
            y_true = torch.zeros([nr_langs*batch_size])#.long()
            
            y_true[0:batch_size] = real_label  # First elements are target embeddings

            
            # All-real minibatch
            x_real = x[languages['trgt'][0]]  # Extract all-real data
            y_preds[0:batch_size] = net.discriminator(x_real)
            
            # All-fake minibatches - One minibatch per source language
            for i, language in enumerate(languages['src']):
                idx_from = batch_size*i+batch_size*nr_trgt_langs
                idx_to = batch_size*(i+1)+batch_size*nr_trgt_langs
                x_trans = net.generator(x[language], language)  # Generate fake data aka translate
                y_preds[idx_from:idx_to] = net.discriminator(x_trans)
            #print('Preds:', y_preds)
            
            # Loss proportional to discriminator's probability of correctly distinguishing TP and FP
            loss = criterion(y_preds, y_true)
            loss.backward()    # Compute gradients only for discriminator
            loss_dis += loss
            
            # Weight update for discriminator
            optimizers['dis'].step() 

            
            # Update generator #
            net.generator.train()
            net.discriminator.eval()
            net.generator.zero_grad()
            
            # Retrieve data
            x = get_train_data(languages, vocabs, batch_size)#.to(device)
            
            # All-fake minibatches - One minibatch per source language
            y_true = torch.full((batch_size,), real_label)#.long()#.to(device) # Try to fool the discriminator
            for language in languages['src']:
                x_src = x[language]
                x_trans = net.generator(x_src, language)
                y_pred = net.discriminator(x_trans)
                # Loss proportional to discriminator's probability of misclassifying TP and FP
                loss = criterion(y_pred, y_true)
                loss.backward()    # Compute gradients only for discriminator
                loss_gen += loss
            
            
            if avg_grads:
                for p in net.generator.decoder.parameters():
                    p.grad *= avg_factor
                    
            
            # Perform weight updates
            for language in languages['src']:
                optimizers['gen'][language].step()
        
        
            ### AUTO ENCODER STEP ###

            # Retrieve data
            x = get_train_data(languages, vocabs, batch_size)#.to(device) 

            # Perform training
            ae.train()
            optimizer_ae.zero_grad()
            train_data = x[languages['trgt'][0]]
            y_pred = ae(train_data)
            loss = AutoEncoderLoss(y_pred, train_data)
            loss.backward()
            optimizer_ae.step()
            loss_ae += loss
            #print('Loss AE:', loss.detach().numpy())
            
            
            ### NOISY AUTO ENCODER STEP ###
            
            # Retrieve data
            x = get_train_data(languages, vocabs, batch_size)#.to(device)
            
            # Perform training
            for lang in languages['src']:
                nae[lang].train()
                nae[lang].zero_grad()
                y_true = x[lang]
                y_pred = nae[lang](y_true)
                loss = AutoEncoderLoss(y_pred, y_true)
                loss.backward()
                optimizers_nae[lang].step()
                loss_nae += loss
                #print('Loss NAE:', loss.detach().numpy())
        
        
        # Document accumulated losses per epoch
        train_loss_gen.append(loss_gen.detach().numpy())
        train_loss_dis.append(loss_dis.detach().numpy())
        train_loss_ae.append(loss_ae.detach().numpy())
        train_loss_nae.append(loss_nae.detach().numpy())
        
        #print('Mean: ', mean_param(net.generator.decoder))
        print('Progress: ', loss_gen.detach().numpy(), 
                            loss_dis.detach().numpy(), 
                            loss_ae.detach().numpy(),
                            loss_nae.detach().numpy())
        
        print('Summed abs weights Generator:', get_summed_abs_grads(net.generator).detach().numpy())
        print('Summed abs weights Discrimi.:', get_summed_abs_grads(net.discriminator).detach().numpy())
        
        # Evaluation step
        if epoch > 0 and epoch % eval_frequency is 0:
            evaluation(net.generator, languages, source_vocabs, eval_words, source_full_vocabs, target_full_vocabs, dictionaries, neighbors, N)
        
        if early_stop: # if early stopping is enabled or not
            if es.step(loss_gen.detach()): # using the real loss of the generator for now, maybe use something else later? e.g. evaluation loss?
                print('early stopping')
                break  # early stop criterion is met, stop the loop now
        
    # Store model
    torch.save(net.state_dict(), final_state_path + 'final_model%d.pt' % epoch)
    
    # Some provision for final eval
    evaluation(net.generator, languages, source_vocabs, eval_words, source_full_vocabs, target_full_vocabs, dictionaries, neighbors, N)

    
if __name__ == "__main__":
    # execute only if run as a script
    main()
    print('Done.')



Nr source languages: 2
Nr target languages: 1

 {'src': ['de', 'nl'], 'trgt': ['en']}
GAN(
  (generator): Generator(
    (decoder): FeedForwardDecoder(
      (w1): Linear(in_features=300, out_features=300, bias=True)
      (w2): Linear(in_features=300, out_features=300, bias=True)
    )
  )
  (discriminator): Discriminator(
    (w1): Linear(in_features=300, out_features=2048, bias=True)
    (w2): Linear(in_features=2048, out_features=1, bias=True)
    (sigmoid): Sigmoid()
  )
)
Epoch  0 / 100
Progress:  1921.0887 78.25907 3.0251207 16.14736
Summed abs weights Generator: 4050.963
Summed abs weights Discrimi.: 14245.395
Epoch  1 / 100
Progress:  1606.8867 101.3544 2.556398 15.448789
Summed abs weights Generator: 3973.2688
Summed abs weights Discrimi.: 10423.962
Epoch  2 / 100
Progress:  1543.7866 111.17978 2.5042558 14.845287
Summed abs weights Generator: 3948.3984
Summed abs weights Discrimi.: 9129.099
Epoch  3 / 100
Progress:  1441.8309 119.74 2.523792 14.328547
Summed abs weights Gene